In [3]:
import numpy as np
from scipy.integrate import solve_bvp
from bokeh.io import output_notebook, show, curdoc
from bokeh.layouts import column
from bokeh.plotting import figure

# Setting the theme to dark_minimal
curdoc().theme = 'dark_minimal'


# Bloch Theorem

### Theoretical Background: Bloch's Theorem and Electron Wavefunctions in Crystals

#### 1. **Bloch's Theorem**
In crystallography, Bloch's theorem plays a pivotal role. It states that an electron's behavior within a crystal is modulated by the periodic external potential set by the lattice structure. In this context, the wavefunction of the electrons in a crystal, denoted as $\psi$, can be formulated as the product of two parts:
- A simple plane wave, represented as $e^{i\vec{k}\vec{x}}$, which spans the entirety of the lattice space.
- A periodic function, represented as $\phi(x)$, determined by analyzing just a single atomic site, taking periodic boundary conditions into account.

#### 2. **External Potential Near Atomic Positions**
Our journey begins with defining a function that represents the external potential felt by electrons near equilibrium atomic positions. To describe this potential, we employ the Dirac delta function (or simply delta function) with a width "w" and an amplitude "A". The negative amplitude points to the attractive nature of interactions between atoms and electrons.

#### 3. **Partitioning the Wavefunction**
With knowledge of the external potential and by applying the Schrödinger equation to our wavefunction representation, we arrive at a Schrödinger-like second-order differential equation. This equation can be harnessed to solve for the periodic part of the electron wavefunction.

#### 4. **Solving the Wavefunction Equation**
To solve this equation, an initial step is a variable substitution, transforming the second-order equation into its first-order counterpart. Following this, a function is defined to yield the right-hand side of the derivatives for our target function. In the process, our energy reference is set to zero. Boundary conditions are also imposed.

#### 5. **The Plane Wave and Constructing the Full Wavefunction**
Post determining $\phi(x)$, the other facet of the Bloch wavefunction is a straightforward plane wave. Merging both parts of the wavefunction as $\psi(x) = e^{i\vec{k}\vec{x}}\phi(x)$, we gain a comprehensive representation of the electron's wavefunction in the crystal.


In [4]:
# Define the potential function as a Dirac delta-like function
def delta(x, w, A):
    return A / (w * np.sqrt(np.pi)) * np.exp(-x**2 / w**2)

def potential(x):
    return delta(x, w, A)

# Define the Bloch function to calculate the behavior of electrons in a crystal
def bloch(x, u):
    k = 2 * np.pi / (2 * w)
    return np.vstack((u[1], (k**2 + delta(x, w, A)) * u[0] + 2 * 1j * k * u[1]))

# Define boundary conditions for the Bloch function
def bc(x, u):
    return np.array([x[0], u[0]])

# Set initial parameters for the potential and the range of x values

s = 10  # Crystallography period
x = np.linspace(-s, s, 100)
w = 3  # ionic potential (delta function) width
A = -20  # ionic potential (delta function) amplitude

# Solve the Bloch function for given boundary conditions
u0 = np.zeros((2, x.size))
u0[0] = 1
ws = solve_bvp(bloch, bc, x, u0)
phi = ws.sol(x)[0]

x1 = np.linspace(-4 * s, 4 * s, 2000)
k = 2 * np.pi / (2 * w)

# Define a plane wave function
def py(x):
    return np.exp(1j * k * x)

# Using Bokeh for plotting
output_notebook()

# First figure: phi
# Plot the external potential near an atomic position
p1 = figure(width=800, height=400, title="Potential and Phi")
p1.line(x, np.real(phi), line_color="white", legend_label="Phi", line_width=3)
p1.circle(0, min(potential(x)), color="blue", size=10, legend_label="Atom")
p1_extra = p1.line(x, potential(x), line_color="red", legend_label="Potential")

show(p1)

/opt/jlab-env/lib/python3.11/site-packages/scipy/integrate/_bvp.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(fun(x, y), dtype)


Loading BokehJS ...

In [5]:
# Second figure: Planewave (Single Site)
# Plot the probability distribution along with the external potential and atomic site

p2 = figure(width=800, height=400, title="Planewave (Single Site)")
p2.line(x, np.real(py(x)), line_color="white", legend_label="Plane Wave", line_width=3)
p2.circle(0, min(potential(x)), color="red", size=10, legend_label="Atom")
p2_extra = p2.line(x, potential(x), line_color="blue", legend_label="Potential")


show(p2)

In [6]:
# Second figure

p2 = figure(width=800, height=400)
p2.line(x, np.real(py(x)), line_color="blue", legend_label="Plane Wave (Single Site)", line_width=3)
p2.circle(0, 0, color="red", size=10, legend_label="Atom")
p2_extra = p2.line(x, potential(x), line_color="red")

p3 = figure(width=800, height=400)
p3.line(x1, np.real(py(x1)), line_color="blue", legend_label="Plane Wave (Multiple Sites)", line_width=3)
p3.circle(np.arange(np.min(x1),np.max(x1)+s,s), [0]*(int(np.ptp(x1)/s)+1), color="red", size=5, legend_label="Atoms")
p3_extra = p3.line(x1, potential((x1-s/2)%s-s/2), line_color="red")

show(p2)



In [7]:
# Third figure: Planewave (Multiple Sites)
p3 = figure(width=800, height=400, title="Planewave (Multiple Sites)")
p3.line(x1, np.real(py(x1)), line_color="white", legend_label="Plane Wave", line_width=3)
p3.circle(np.arange(np.min(x1), np.max(x1) + s, s), [min(potential(x1))]*(int(np.ptp(x1)/s) + 1), color="red", size=5, legend_label="Atoms")
p3_extra = p3.line(x1, potential((x1 - s/2) % s - s/2), line_color="blue", legend_label="Potential")

show(p3)

In [8]:
# Fourth figure: Modulated Wavefunction (Single Site)
# Construct the full electron wavefunction and plot it
p4 = figure(width=800, height=400, title="Modulated Wavefunction (Single Site)")
psi0 = py(x) * ws.sol(x)[0]
p4.line(x, np.real(psi0), line_color="white", legend_label="Modulated Wavefunction", line_width=3)
p4.circle(0, min(potential(x)), color="red", size=10, legend_label="Atom")
p4_extra = p4.line(x, potential(x), line_color="blue", legend_label="Potential")

show(p4)

In [7]:
# Fifth figure: Wavefunction (Multiple Sites)
# Plot the electron wavefunction across multiple atomic sites
p5 = figure(width=800, height=400, title="Wavefunction (Multiple Sites)")
psi1 = py(x1) * ws.sol((x1 - s/2) % s - s/2)[0]
p5.line(x1, np.real(psi1), line_color="white", legend_label="Wavefunction", line_width=3)
p5.circle(np.arange(np.min(x1), np.max(x1) + s, s), [min(potential(x1))]*(int(np.ptp(x1)/s) + 1), color="red", size=5, legend_label="Atoms")
p5_extra = p5.line(x1, potential((x1 - s/2) % s - s/2), line_color="blue", legend_label="Potential")

show(p5)

**1. Bloch Theorem Basics:**

- **How does Bloch's theorem describe the wavefunction of an electron in a periodic potential?**
    - How does Bloch's theorem describe the wavefunction $\psi(x)$ of an electron in a periodic potential?

- **According to the given description, the wavefunction can be represented as $\psi(x) = e^{i\vec{k}\vec{x}}\phi(x)$. Break down each term in this expression and explain its significance.**
    - According to the given description, the wavefunction can be represented as $\psi(x) = e^{i\vec{k}\vec{x}}\phi(x)$. Break down each term in this expression and explain its significance.

**2. External Potential:**

- **How does the Dirac Delta function, described with a width "w" and amplitude "A", approximate the external potential felt by electrons near atomic positions?**
    - How does the Dirac Delta function, described with a width "w" and amplitude "A", approximate the external potential felt by electrons near atomic positions?

**3. Mathematical Analysis:**

- **Considering the Schrodinger equation, how is the electron wavefunction $\psi$ affected by the periodic external potential?**
    - Considering the Schrodinger equation, how is the electron wavefunction $\psi$ affected by the periodic external potential?

- **Given that we use the Dirac Delta function to describe the potential, how is the width $w$ and amplitude $A$ related to the physical properties of the system?**
    - Given that we use the Dirac Delta function to describe the potential, how is the width $w$ and amplitude $A$ related to the physical properties of the system?

**4. Implementation Insights:**

- **In the given code, how is the Schrodinger-like equation solved numerically for the periodic function $\phi(x)$?**
    - In the given code, how is the Schrodinger-like equation solved numerically for the periodic function $\phi(x)$?

- **Discuss the significance of boundary conditions in solving the differential equations for $\phi(x)$ and how they are applied in the context of this code.**
    - Discuss the significance of boundary conditions in solving the differential equations for $\phi(x)$ and how they are applied in the context of this code.

**5. Advanced Understanding:**

- **Using the expression $\psi(x) = e^{i\vec{k}\vec{x}}\phi(x)$, discuss how varying the wave vector $\vec{k}$ might influence the electron's behavior in the crystal.**
    - Using the expression $\psi(x) = e^{i\vec{k}\vec{x}}\phi(x)$, discuss how varying the wave vector $\vec{k}$ might influence the electron's behavior in the crystal.

- **If a different potential function was used instead of the Dirac Delta approximation, how might that change the nature and properties of $\phi(x)$ and subsequently $\psi(x)$?**
    - If a different potential function was used instead of the Dirac Delta approximation, how might that change the nature and properties of $\phi(x)$ and subsequently $\psi(x)$?
